<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Self_Supervised_Learning_for_Label_Free_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np

# Define self-supervised learning model using contrastive learning
class SSLModel(nn.Module):
    def __init__(self, base_model):
        super(SSLModel, self).__init__()
        self.encoder = base_model
        self.projection = nn.Linear(self.encoder.fc.in_features, 128)
        self.encoder.fc = nn.Identity()  # Remove final classifier layer

    def forward(self, x1, x2):
        z1 = self.projection(self.encoder(x1))
        z2 = self.projection(self.encoder(x2))
        return z1, z2

# Contrastive loss for self-supervised learning
def contrastive_loss(z1, z2, temperature=0.5):
    sim = torch.nn.functional.cosine_similarity(z1.unsqueeze(1), z2.unsqueeze(0), dim=2)
    loss = -torch.log(torch.exp(sim / temperature).diag() / torch.exp(sim / temperature).sum(1))
    return loss.mean()

# Example usage
model = SSLModel(models.resnet18(pretrained=True))
image_augmentations = transforms.Compose([transforms.RandomResizedCrop(224), transforms.ToTensor()])

# Create dummy images as PIL images
image1 = Image.fromarray(np.uint8(np.random.rand(224, 224, 3) * 255))
image2 = Image.fromarray(np.uint8(np.random.rand(224, 224, 3) * 255))

image1 = image_augmentations(image1)
image2 = image_augmentations(image2)

z1, z2 = model(image1.unsqueeze(0), image2.unsqueeze(0))
loss = contrastive_loss(z1, z2)
print("Self-supervised contrastive loss:", loss.item())